In [1]:
import pandas as pd 
import csv  

# Import Kaggle CSV With Cases & Deaths By State #

In [3]:
# Import Kaggle csv file with total cases and deaths by US state and county
# Data was downloaded from https://www.kaggle.com/imdevskp/corona-virus-report#usa_county_wise.csv
file = "Raw_Data/usa_county_wise.csv"
kaggle_df = pd.read_csv(file)
kaggle_df.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed,Deaths
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,"American Samoa, US",1/22/20,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,"Guam, US",1/22/20,0,0
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,"Northern Mariana Islands, US",1/22/20,0,0
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,"Puerto Rico, US",1/22/20,0,0
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,"Virgin Islands, US",1/22/20,0,0


In [4]:
# Only keep rows for US states (i.e. remove Guam, Virgin Islands, etc.)
kaggle_us = kaggle_df.loc[kaggle_df["iso2"] == "US",:]

In [5]:
# Remove columns not needed (UID, iso2(country/territory), iso3(country/territory), & code3)
kaggle_rem_cols = kaggle_us[["FIPS", "Admin2","Province_State", "Lat", "Long_", "Combined_Key", "Date", "Confirmed", "Deaths"]]

In [12]:
# Rename columns (Admin 2 to County, Province_State to State, Long_ to Lng)
kaggle_renamed = kaggle_rem_cols.rename(columns={"Admin2":"County", "Province_State":"State", "Long_":"Lng"})
kaggle_renamed.sample(10)

,FIPS,County,State,Lat,Lng,Combined_Key,Date,Confirmed,Deaths
277212,13295.0,Walker,Georgia,34.733764,-85.301652,"Walker, Georgia, US",4/16/20,13,0
156876,17073.0,Henry,Illinois,41.353399,-90.132962,"Henry, Illinois, US",3/10/20,0,0
220323,41051.0,Multnomah,Oregon,45.547480,-122.416935,"Multnomah, Oregon, US",3/29/20,91,2
252575,37091.0,Hertford,North Carolina,36.360776,-76.981041,"Hertford, North Carolina, US",4/8/20,5,1
164223,28135.0,Tallahatchie,Mississippi,33.953399,-90.172940,"Tallahatchie, Mississippi, US",3/12/20,0,0
188663,56009.0,Converse,Wyoming,42.972723,-105.508185,"Converse, Wyoming, US",3/19/20,0,0
19000,48395.0,Robertson,Texas,31.025396,-96.512722,"Robertson, Texas, US",1/27/20,0,0
110744,2020.0,Anchorage,Alaska,61.149982,-149.142699,"Anchorage,Alaska,US",2/25/20,0,0
214746,80026.0,Out of MI,Michigan,0.000000,0.000000,"Out of MI, Michigan, US",3/27/20,8,0
87763,56019.0,Johnson,Wyoming,44.040572,-106.584517,"Johnson, Wyoming, US",2/17/20,0,0


In [11]:
# Check for missing values
kaggle_renamed.count()

FIPS            285648
County          285824
State           286000
Lat             286000
Lng             286000
Combined_Key    286000
Date            286000
Confirmed       286000
Deaths          286000
dtype: int64

In [13]:
# Drop rows with missing data
kaggle_drop = kaggle_renamed.dropna()
kaggle_drop.count()

FIPS            285472
County          285472
State           285472
Lat             285472
Lng             285472
Combined_Key    285472
Date            285472
Confirmed       285472
Deaths          285472
dtype: int64

In [14]:
# Check if date column date are strings or date objects
kaggle_drop.dtypes

FIPS            float64
County           object
State            object
Lat             float64
Lng             float64
Combined_Key     object
Date             object
Confirmed         int64
Deaths            int64
dtype: object

In [15]:
# Change dates from strings to datetime objects
kaggle_drop['Date'] = pd.to_datetime(kaggle_drop['Date'],format='%m/%d/%y')

In [16]:
# Check that column type change
kaggle_drop.dtypes

FIPS                   float64
County                  object
State                   object
Lat                    float64
Lng                    float64
Combined_Key            object
Date            datetime64[ns]
Confirmed                int64
Deaths                   int64
dtype: object

In [18]:
# Export kaggle df as a csv so it can be imported to postgres
kaggle_drop.to_csv("counties_cases_deaths.csv", encoding="utf-8", index=False)

# Import CDC CSV With COVID Forecasts By State #

In [13]:
# Import csv file with forecast of potential deaths by state
# Data was downloaded from https://www.cdc.gov/coronavirus/2019-ncov/covid-data/forecasting-us.html
file2 = "Raw_Data/forecast_data_0413.csv"
forecast_raw = pd.read_csv(file2)
forecast_raw.tail()

,model,forecast_date,target,target_week_end_date,location_name,point,quantile_0.025,quantile_0.975
1325,MOBS,4/13/2020,2 wk ahead cum death,4/25/2020,Washington,939,NaN,NaN
1326,MOBS,4/13/2020,2 wk ahead cum death,4/25/2020,Wisconsin,199,NaN,NaN
1327,MOBS,4/13/2020,2 wk ahead cum death,4/25/2020,West Virginia,13,NaN,NaN
1328,MOBS,4/13/2020,2 wk ahead cum death,4/25/2020,Wyoming,15,NaN,NaN
1329,MOBS,4/13/2020,2 wk ahead cum death,4/25/2020,US,46548,NaN,NaN


In [15]:
# Only keep rows for states, not whole of US
forecast_states = forecast_raw.loc[forecast_raw["location_name"] != "US",:]
forecast_states.head()

,model,forecast_date,target,target_week_end_date,location_name,point,quantile_0.025,quantile_0.975
4,ensemble forecast,4/13/2020,1 wk ahead cum death,4/18/2020,Alabama,152,102.0,325.0
5,ensemble forecast,4/13/2020,2 wk ahead cum death,4/25/2020,Alabama,243,116.0,676.0
6,ensemble forecast,4/13/2020,3 wk ahead cum death,5/2/2020,Alabama,339,131.0,1151.0
7,ensemble forecast,4/13/2020,4 wk ahead cum death,5/9/2020,Alabama,428,142.0,1836.0
8,ensemble forecast,4/13/2020,1 wk ahead cum death,4/18/2020,Delaware,52,36.0,146.0


In [17]:
# Drop rows with missing data
forecast_states = forecast_states.dropna()
forecast_states.count()

model                   1220
forecast_date           1220
target                  1220
target_week_end_date    1220
location_name           1220
point                   1220
quantile_0.025          1220
quantile_0.975          1220
dtype: int64

In [32]:
# Rename columns (target week end date to target end date, location name to state, point to actual)
forecast_renamed = forecast_states.rename(columns={"target_week_end_date":"target_end_date", "location_name":"state", "point":"actual"})
forecast_renamed.head(20)

,model,forecast_date,target,target_end_date,state,actual,quantile_0.025,quantile_0.975
4,ensemble forecast,4/13/2020,1 wk ahead cum death,4/18/2020,Alabama,152,102.0,325.0
5,ensemble forecast,4/13/2020,2 wk ahead cum death,4/25/2020,Alabama,243,116.0,676.0
6,ensemble forecast,4/13/2020,3 wk ahead cum death,5/2/2020,Alabama,339,131.0,1151.0
7,ensemble forecast,4/13/2020,4 wk ahead cum death,5/9/2020,Alabama,428,142.0,1836.0
8,ensemble forecast,4/13/2020,1 wk ahead cum death,4/18/2020,Delaware,52,36.0,146.0
9,ensemble forecast,4/13/2020,2 wk ahead cum death,4/25/2020,Delaware,71,36.0,295.0
10,ensemble forecast,4/13/2020,3 wk ahead cum death,5/2/2020,Delaware,90,37.0,496.0
11,ensemble forecast,4/13/2020,4 wk ahead cum death,5/9/2020,Delaware,108,38.0,828.0
12,ensemble forecast,4/13/2020,1 wk ahead cum death,4/18/2020,District of Columbia,75,54.0,154.0
13,ensemble forecast,4/13/2020,2 wk ahead cum death,4/25/2020,District of Columbia,103,59.0,272.0


In [24]:
# Check if date column types are datetime or string 
forecast_renamed.dtypes

model               object
forecast_date       object
target              object
target_end_date     object
state               object
actual               int64
quantile_0.025     float64
quantile_0.975     float64
dtype: object

In [28]:
# Change dates from strings to datetime objects
forecast_renamed['forecast_date'] = pd.to_datetime(forecast_renamed['forecast_date'],format='%m/%d/%Y')
forecast_renamed['target_end_date'] = pd.to_datetime(forecast_renamed['target_end_date'],format='%m/%d/%Y')

In [29]:
# Verify columns changed to datetime
forecast_renamed.dtypes

model                      object
forecast_date      datetime64[ns]
target                     object
target_end_date    datetime64[ns]
state                      object
actual                      int64
quantile_0.025            float64
quantile_0.975            float64
dtype: object

In [30]:
# Export forecast df as a csv so it can be imported to postgres
forecast_renamed.to_csv("forecast.csv", encoding="utf-8", index=False)